# Función principal
### Data set para almacenar todos los datos disponibles en el api 

In [1]:
# import urllib library
from urllib.request import urlopen
import pandas as pd
import json
import os
from datetime import datetime

os.environ['URL'] = 'https://covid-api.mmediagroup.fr/v1/cases'
# store the URL in url as 
# parameter for urlopen
# url = "https://covid-api.mmediagroup.fr/v1/cases"

url = os.getenv('URL')

def data_download():
    """Primary function to extract data from Covid API and download to Google Storage
    """
  
    # store the response of URL
    response = urlopen(url)
    
    # storing the JSON response 
    # from url in data
    data_json = json.loads(response.read())
    data_items = data_json.items()

    # convert json to dataframe
    data_list = list(data_items)
    df = pd.DataFrame(data_list)
    print(df)
    #filter dataframe
    df_filtrate = df.join(pd.json_normalize(df[1])).fillna(0)
    df_filtrate = df_filtrate.drop(['All.country'], axis =1)
    df_filtrate = df_filtrate.rename(columns={0:'All.country'})
    # list with columns name 
    list_name = df_filtrate.columns.to_list()

    # filter columns name with relevant columns
    new_lis_name = [str(i) for i in list_name if 'All' in str(i)]

    # Datafraeme only relevant columns
    df_filtrate_by_all = df_filtrate[new_lis_name]
    # filter dataframe
    new_list_name_columns = df_filtrate_by_all.columns.to_list()
    new_list_name_without_all = [str(i).split('.')[1] for i in new_list_name_columns]
    df_filtrate_by_all.columns = new_list_name_without_all

    # # export json and csv with date of dowload
    today = str(datetime.date(datetime.now()))
    df_filtrate_by_all.to_json('data_download_covid-'+today+'.json', orient="table")
    df_filtrate_by_all.to_csv('data_download_covid-'+today+'.csv', index=False)

    return df_filtrate_by_all

df = data_download()
df

                      0                                                  1
0           Afghanistan  {'All': {'confirmed': 157858, 'recovered': 0, ...
1               Albania  {'All': {'confirmed': 203787, 'recovered': 0, ...
2               Algeria  {'All': {'confirmed': 212848, 'recovered': 0, ...
3               Andorra  {'All': {'confirmed': 19440, 'recovered': 0, '...
4                Angola  {'All': {'confirmed': 65404, 'recovered': 0, '...
..                  ...                                                ...
192  West Bank and Gaza  {'All': {'confirmed': 463573, 'recovered': 0, ...
193               Yemen  {'All': {'confirmed': 10069, 'recovered': 0, '...
194              Zambia  {'All': {'confirmed': 211077, 'recovered': 0, ...
195            Zimbabwe  {'All': {'confirmed': 167140, 'recovered': 0, ...
196              Global  {'All': {'population': 7444509223, 'confirmed'...

[197 rows x 2 columns]


,country,confirmed,recovered,deaths,population,sq_km_area,life_expectancy,elevation_in_meters,continent,abbreviation,location,iso,capital_city,lat,long,updated
0,Afghanistan,157858,0,7328,3.553008e+07,652090.0,45.9,0,Asia,AF,Southern and Central Asia,4.0,Kabul,33.93911,67.709953,2021-12-13 04:22:02
1,Albania,203787,0,3134,2.930187e+06,28748.0,71.6,0,Europe,AL,Southern Europe,8.0,Tirana,41.1533,20.1683,2021-12-13 04:22:02
2,Algeria,212848,0,6145,4.131814e+07,2381741.0,69.7,800,Africa,DZ,Northern Africa,12.0,Alger,28.0339,1.6596,2021-12-13 04:22:02
3,Andorra,19440,0,133,7.696500e+04,468.0,83.5,"1,996",Europe,AD,Southern Europe,20.0,Andorra la Vella,42.5063,1.5218,2021-12-13 04:22:02
4,Angola,65404,0,1737,2.978419e+07,1246700.0,38.3,"1,112",Africa,AO,Central Africa,24.0,Luanda,-11.2027,17.8739,2021-12-13 04:22:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,West Bank and Gaza,463573,0,4830,0.000000e+00,0.0,0,0,0,0,0,0.0,0,31.9522,35.2332,2021-12-13 04:22:02
193,Yemen,10069,0,1971,2.825042e+07,527968.0,59.8,999,Asia,YE,Middle East,887.0,Sanaa,15.552726999999999,48.516388,2021-12-13 04:22:02
194,Zambia,211077,0,3670,1.709413e+07,752618.0,37.2,"1,138",Africa,ZM,Eastern Africa,894.0,Lusaka,-13.133897,27.849332,2021-12-13 04:22:02
195,Zimbabwe,167140,0,4738,1.652990e+07,390757.0,37.8,961,Africa,ZW,Eastern Africa,716.0,Harare,-19.015438,29.154857,2021-12-13 04:22:02


# Función # 2
### Datos del día de hoy 

In [35]:
# import urllib library
from urllib.request import urlopen
import pandas as pd
import json
import os
from datetime import datetime

def today_data():
    """Function to dowload file current day if it exists
    """
    today = str(datetime.date(datetime.now()))
    file_path = 'data_download_covid-'+today+'.csv'
    try:
        df = pd.read_csv(file_path)
        list_today_data = ['country', 'confirmed', 'deaths', 'location']
        df_today_data = df[list_today_data]

        # # export json and csv
        df_today_data.to_json('today_data_covid.json', orient="table")
        df_today_data.to_csv('today_data_covid.csv', index=False)
        return df_today_data
    except:
        print('Error, File not foud')

df = today_data()
df

,country,confirmed,deaths,location
0,Afghanistan,156896,7365,Southern and Central Asia
1,Albania,195988,3035,Southern Europe
2,Algeria,208952,6021,Northern Africa
3,Andorra,16086,130,Southern Europe
4,Angola,65024,1730,Central Africa
...,...,...,...,...
192,Yemen,9955,1938,Middle East
193,Zambia,210070,3667,Eastern Africa
194,Zimbabwe,133638,4699,Eastern Africa
195,US,47730591,771118,North America


# Función # 3
### Datos agrupados por “Location”,

In [7]:
# import urllib library
from urllib.request import urlopen
import pandas as pd
import json
import os
from datetime import datetime

def location_data():
    """Function to dowload file group by location
    """
    today = str(datetime.date(datetime.now()))
    file_path = 'data_download_covid-'+today+'.csv'
    try:
        df = pd.read_csv(file_path)
        list_today_data = ['confirmed', 'deaths', 'location']
        df_location_group = df[list_today_data]
        df_location_group = df_location_group.groupby('location')
        df_location_group = df_location_group.sum().reset_index()

        # # export json and csv
        df_location_group.to_json('location_group.json', orient="table")
        df_location_group.to_csv('location_group.csv', index=False)
        return df_location_group
    except:
        print('Error, File not foud')

df = location_data()
df

,location,confirmed,deaths
0,0,260801630,5208920
1,Australia and New Zealand,209836,1988
2,Baltic Countries,919055,12206
3,British Isles,10426170,149978
4,Caribbean,1628054,19120
5,Central Africa,242301,4275
6,Central America,6059559,337725
7,Eastern Africa,1602201,33155
8,Eastern Asia,2632434,28430
9,Eastern Europe,23511929,600903


# Función # 4
### Datos con índice de muertes por km**2

In [17]:
# import urllib library
from urllib.request import urlopen
import pandas as pd
import json
import os
from datetime import datetime

def death_rate():
    """Function to dowload file with a new column to death rate in % 
    """
    today = str(datetime.date(datetime.now()))
    file_path = 'data_download_covid-'+today+'.csv'
    try:
        df = pd.read_csv(file_path)
        # make new column
        df['death_rate (%)'] = (df['deaths']/df['sq_km_area'])*100
        list_today_data = ['country', 'deaths', 'death_rate (%)']
        df = df[list_today_data]

        # # export json and csv
        df.to_json('death_rate.json', orient="table")
        df.to_csv('death_rate.csv', index=False)
        return df
    except:
        print('Error, File not foud')

df = death_rate()
df

,country,deaths,death_rate (%)
0,Afghanistan,7365,1.129445
1,Albania,3035,10.557256
2,Algeria,6021,0.252798
3,Andorra,130,27.777778
4,Angola,1730,0.138766
...,...,...,...
192,Yemen,1938,0.367068
193,Zambia,3667,0.487233
194,Zimbabwe,4699,1.202538
195,US,771118,8.235343
